## Read namespaces

In [ ]:
# Read namespaces
import os
import kubernetes
import ipywidgets as widgets
from IPython.display import display, JSON

ns_list = []
kubernetes.config.load_kube_config()
k8s_client = kubernetes.client.CoreV1Api()
namespaces = k8s_client.list_namespace(_request_timeout=(1, 5))
for namespace in namespaces.items:
    ns_name = namespace.metadata.name
    ns_list.append(ns_name)
w = widgets.Dropdown(
    options=ns_list,
    description='SUT ns',
    disabled=False,
    layout=widgets.Layout(width='50%')
)
display(w)

## Set namespaces

In [ ]:
def show_textbox(box_value, box_desc):
    textbox_w = widgets.Text(
        value=box_value,
        description=box_desc,
        disabled=True,
        layout=widgets.Layout(width='60%')
    )
    display(textbox_w)

# Set namespaces
SUT_NAMESPACE = w.value
# print(f"Using SUT namespace {SUT_NAMESPACE}")
BRANCH_NAME = SUT_NAMESPACE.replace("ci-ska-mid-itf-", "")
# print(f"Using SUT branch {BRANCH_NAME}")
SKA001_NAMESPACE = f"ci-dish-lmc-ska001-{BRANCH_NAME}"

show_textbox(SKA001_NAMESPACE, 'SKA001 ns')

SKA036_NAMESPACE = f"ci-dish-lmc-ska036-{BRANCH_NAME}"

show_textbox(SKA036_NAMESPACE, 'SKA036 ns')

# print(f"Setting up parameters for SUT namespace {SUT_NAMESPACE}\n")

TANGO_HOST = f"tango-databaseds.{SUT_NAMESPACE}.svc.miditf.internal.skao.int:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

show_textbox(os.getenv("TANGO_HOST"), 'Tango host')

## Device Proxies

### Device Proxy Names

| Class     | Description             | Name                                |
|---------- | ----------------------- | ----------------------------------- |
| TMC       | tmc central node        | ska_mid/tm_central/central_node     |
|           | tmc csp master          | ska_mid/tm_leaf_node/csp_master     |
|           | tmc csp subarray        | ska_mid/tm_leaf_node/csp_subarray01 |
|           | tmc subarray            | ska_mid/tm_subarray_node/1          |
| CSP LMC   | csp control             | mid-csp/control/0                   |
|           | csp subarray            | mid-csp/subarray/01                 |
| CBF       | cbf controller          | mid_csp_cbf/sub_elt/controller      |
|           | cbf subarray            | mid_csp_cbf/sub_elt/subarray_01     |
| Dish Leaf | dish leaf node ska001   | ska_mid/tm_leaf_node/d0001          |
|           | dish leaf node ska036   | ska_mid/tm_leaf_node/d0036          |
| SDP       | sdp subarray            | mid-sdp/subarray/01                 |
|           | sdp_subarray_leaf_node  | ska_mid/tm_leaf_node/sdp_subarray01 |
| CSP       | csp_subarray_leaf_node  | ska_mid/tm_leaf_node/csp_subarray01 |
|           | csp_master_leaf_node    | ska_mid/tm_leaf_node/csp_master     |


### Configure Device Proxies

In [ ]:
# Set Up and Configure Device Proxies

RECEPTORS = ["SKA001", "SKA036"]

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/mid_telescope/tmc"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

CBF_CONFIGS = f"{DATA_DIR}/mid_telescope/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

# print("Links for Taranta and QA Display\n")
print("SUT Links")
print(f"https://k8s.miditf.internal.skao.int/{SUT_NAMESPACE}/signal/display/")
print(f"https://k8s.miditf.internal.skao.int/{SUT_NAMESPACE}/taranta/devices")
print("\n")
print("Dish LMC links")
print(f"https://k8s.miditf.internal.skao.int/{SKA001_NAMESPACE}/taranta/devices")
print(f"https://k8s.miditf.internal.skao.int/{SKA036_NAMESPACE}/taranta/devices")

### Set Up Device Proxies

In [ ]:
import json
import time
import tango

from tango import DeviceProxy

proxy_w = widgets.IntProgress(
    value=0,
    min=0,
    max=11,
    description='Proxies',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(proxy_w)
proxy_ok = True
try:
    # TMC proxies
    proxy_w.value = 1
    tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
    proxy_w.value = 2
    tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
    proxy_w.value = 3
    tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
    proxy_w.value = 4
    tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")
    # CSP.LMC proxies
    proxy_w.value = 5
    csp_control = DeviceProxy("mid-csp/control/0")
    proxy_w.value = 6
    csp_subarray = DeviceProxy("mid-csp/subarray/01")
    # CBF proxies
    proxy_w.value = 7
    cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
    proxy_w.value = 8
    cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")
    # Dish Leaf Proxies
    proxy_w.value = 9
    dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")
    proxy_w.value = 10
    dish_leaf_node_ska036 = DeviceProxy("ska_mid/tm_leaf_node/d0036")
    # SDP Proxies
    proxy_w.value = 11
    sdp_subarray = DeviceProxy("mid-sdp/subarray/01")
except tango.DevFailed as terr:
    print(f"ERROR: {terr.args[0].desc.strip()}")
    tmc_central_node = None
    tmc_csp_master = None
    proxy_ok = False
        
if proxy_ok:
    print(f'Set up {proxy_w.value} device proxies')




## Diagnostics

### Smoke test

In [ ]:
# Smoke test
def smoke_test(tango_dev, tango_desc):
    smoke = False
    if tango_dev is None:
        smoke = False
    else:
        try:
            tping = tango_dev.ping()
            # print(f"Ping TMC central node: {tping}")
            smoke = True
        except tango.ConnectionFailed as terr:
            err_msg = terr.args[0].desc.strip()
            print(f"Tango connection failed: {err_msg}")
            smoke = False
    smoke_w = widgets.Valid(
        value=smoke,
        description=f"{tango_desc} online",
        layout=widgets.Layout(width='60%'),
        style = {'description_width': 'initial'}
    )
    display(smoke_w)


#    if tping:
#        tmc_admin = tmc_central_node.adminMode
#    else:
#        tmc_admin = -1
    
 
smoke_test(tmc_central_node, "TMC Central Node")
smoke_test(tmc_csp_master, "TMC CSP master")
smoke_test(tmc_csp_subarray, "TMC CSP subarray"),
smoke_test(tmc_subarray, "TMC subarray"),
smoke_test(csp_control, "CSP control"),
smoke_test(csp_subarray, "CSP subarray"),
smoke_test(cbf_controller, "CBF controller"),
smoke_test(cbf_subarray, "CBF subarray")
smoke_test(dish_leaf_node_ska001, "Dish leaf 001")
smoke_test(dish_leaf_node_ska036, "Dish leaf 036")
smoke_test(sdp_subarray, "SDP subarray")

### Admin mode test

In [ ]:
def admin_test(tango_dev, tango_desc):
    dev_admin = tango_dev.adminMode
    admin_w = widgets.Valid(
        value=not dev_admin,
        description=f"{tango_desc} admin",
        layout=widgets.Layout(width='60%'),
        style = {'description_width': 'initial'}
    )
    display(admin_w)
    
admin_test(tmc_central_node, "TMC Central Node")
admin_test(tmc_csp_master, "TMC CSP master")
admin_test(tmc_csp_subarray, "TMC CSP subarray"),
admin_test(tmc_subarray, "TMC subarray"),
admin_test(csp_control, "CSP control"),
admin_test(csp_subarray, "CSP subarray"),
admin_test(cbf_controller, "CBF controller"),
admin_test(cbf_subarray, "CBF subarray")
admin_test(dish_leaf_node_ska001, "Dish leaf 001")
admin_test(dish_leaf_node_ska036, "Dish leaf 036")
admin_test(sdp_subarray, "SDP subarray")

### Print TMC diagnostics

In [ ]:
# Print TMC diagnostics
try:
    print(f"| TMC Central Node           | {tmc_central_node.name():40} |")
    print(f"| TMC Central Node state     | {str(tmc_central_node.State()):40} |")
    print(f"| TMC Central Node adminMode | {str(tmc_central_node.adminmode):40} |")
    print(f"| TMC Central Node healthState | {str(tmc_central_node.healthState)} |")
    print(f"| TMC Central Node telescopeHealthState | {str(tmc_central_node.telescopeHealthState)} |")
    print(f"| TMC Central Node isDishVccConfig | {str(tmc_central_node.isDishVccConfigSet)} |")
    print(f"| TMC Central Node DishVccValidationStatus | {str(tmc_central_node.DishVccValidationStatus)} |")
    print(f"| TMC Subarray Node state | {tmc_csp_subarray.State()} |")
    print(f"| TMC Subarray adminMode | {str(tmc_csp_subarray.adminMode)} |")
    # print(f"TMC Subarray Node obsState: {str(tmc_csp_subarray.obsState)}")
except tango.ConnectionFailed as terr:
    print(f"Tango connection failed: {terr.args[0].desc.strip()}")
# table(tmc_rows, frame=True)

### Print CSP-LMC Diagnostics

In [ ]:
# Print CSP-LMC Diagnostics
try:
    # Controller
    print(f"| CSP-LMC Controller                     | {csp_control.name():40} |")
    print(f"| CSP-LMC Controller admin mode          | {str(csp_control.adminMode):40} |")
    print(f"| CSP-LMC Controller State               | {csp_control.State():40} |")
    print(f"| CSP-LMC Controller dish Vcc Config     | {csp_control.dishVccConfig:40} |")
    print(f"| CSP-LMC Controller CBF Simulation Mode | {csp_control.cbfSimulationMode:40} |")
    # Subarray
    print(f"| CSP-LMC Subarray                       | {csp_subarray.name():40} |")
    print(f"| CSP-LMC Subarray admin mode            | {str(csp_subarray.adminMode):40} |")
    print(f"| CSP-LMC Subarray State                 | {csp_subarray.State():40} |")
    print(f"| CSP-LMC Subarray observation state     | {str(csp_subarray.obsState):40} |")
    print(f"| CSP-LMC Subarray dish Vcc Config       | {csp_subarray.dishVccConfig:40} |")
except tango.ConnectionFailed as terr:
    print(f"Tango connection failed: {terr.args[0].desc.strip()}")


### Print CBF Diagnostics

In [ ]:
# Print CBF Diagnostics
try:
    # CBF Controller
    print(f"| CBF Controller admin mode      | {str(cbf_controller.adminMode):40} |")
    print(f"| CBF Controller State           | {str(cbf_controller.State()):40} |")
    # CBF Subarray
    print(f"| CBF Subarray admin mode        | {str(cbf_subarray.adminMode):40} |")
    print(f"| CBF Subarray State             | {str(cbf_subarray.State()):40} |")
    print(f"| CBF Subarray observation state | {str(cbf_subarray.obsState):40} |")
except tango.ConnectionFailed as terr:
    err_msg = terr.args[0].desc.strip()
    print("Tango connection failed: %s", err_msg)

### Print SDP Diagnostics

In [ ]:
# Print SDP Diagnostics
try:
    # print(f"SDP Controller state: {sdp_controller.State()}")
    # print(f"SDP Controller admin mode: {str(sdp_controller.admin_mode)}")
    # SDP Subarray
    print(f"SDP Subarray state             | {str(sdp_subarray.State()):40} |")
    print(f"SDP Subarray admin mode        | {str(sdp_subarray.adminMode):40} |")
    print(f"SDP Subarray observation state | {str(sdp_subarray.obsState):40} |")
except tango.ConnectionFailed as terr:
    err_msg = terr.args[0].desc.strip()
    print("Tango connection failed: %s", err_msg)

## Set devices to ONLINE

In [ ]:
# Set devices to adminMode = ONLINE

def set_admin_mode(tango_dev, tango_desc):
    set_admin_w = widgets.Valid(
        value=not tango_dev.adminMode,
        description=f"{tango_desc} online",
        layout=widgets.Layout(width='60%'),
        style = {'description_width': 'initial'}
    )
    display(set_admin_w)

csp_control.adminMode = 0
csp_subarray.adminMode = 0

# Leaf Nodes
csp_subarray_leaf_node_dp = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
sdp_subarray_leaf_node_dp = DeviceProxy("ska_mid/tm_leaf_node/sdp_subarray01")
csp_master_leaf_node_dp = DeviceProxy("ska_mid/tm_leaf_node/csp_master")

time.sleep(2)
print("Admin mode after setting to ONLINE")

set_admin_mode(csp_control, "CSP Control")
set_admin_mode(csp_subarray, "CSP Subarray")
set_admin_mode(cbf_controller, "CBF Controller")
set_admin_mode(cbf_subarray, "CBF Subarray")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

time.sleep(2)
# print("\nChecking CBF Simulation Mode and CBF Timeout:")

# CBF Simulation Mode
# print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")

cbf_sim_w = widgets.Checkbox(
    value=bool(csp_control.cbfSimulationMode),
    description='CBF Simulation Mode',
    disabled=False,
    indent=False
)
display(cbf_sim_w)

# CBF Timeout
# print(f"  CBF Timeout: {csp_control.commandTimeout} sec")
cbf_tim_w = widgets.IntSlider(
    value=csp_control.commandTimeout,
    min=0,
    max=180,
    step=1,
    description='CBF Timeout',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(cbf_tim_w)

## Load the Dish Vcc Config / Init Sys Params

In [ ]:
# Load the Dish Vcc Config / Init Sys Params
from IPython.display import JSON
# csp_control.commandTimeout = cbf_tim_w.value

csp_control.cbfSimulationMode = cbf_sim_w.value

with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][0] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"
# print(f"Dish config json file contents: \n{dish_config_json}")

nmax = 2
cfgp_w = widgets.IntProgress(
    value=0,
    min=0,
    max=nmax,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(cfgp_w)

# NOTE: RUN TWICE
for n in range(1, nmax+1):
    cfgp_w.value = n
    # print(f"\nLoad dish configuration {n}")
    try:
        tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))
        print("Dish config loaded")
    except tango.DevFailed as derr:
        err_msg = derr.args[0].desc.strip()
        print(f"Error: {err_msg}")
    except tango.ConnectionFailed as terr:
        err_msg = terr.args[0].desc.strip()
        print(f"Error: {err_msg}")
    time.sleep(2)

tmc_csp_dish_vcc = json.loads(tmc_csp_master.dishVccConfig)
# print(f"TMC CSP Master's Dish Vcc Config attribute value: \n"
# "{json.dumps(tmc_csp_master.dishVccConfig, indent=4)}")
print(f'TMC CSP Master Dish Vcc Config interface\n{tmc_csp_dish_vcc["interface"]}')
# json.dumps(tmc_csp_master.dishVccConfig, indent=4)
# cfgj = JSON(json.loads(tmc_csp_master.dishVccConfig), expanded=True)
# display(cfgj)

tmc_csp_src_vcc = json.loads(tmc_csp_master.sourceDishVccConfig)
# print(f"TMC CSP Master's Source Dish Vcc Config attribute value: \n"
# "{tmc_csp_master.sourceDishVccConfig}")
print(f'TMC CSP Master Source Dish Vcc Config interface\n{tmc_csp_src_vcc["interface"]}')


## Turn the Telescope On

### Run the Telescope On command

In [ ]:
# Turn the Telescope On
import tango

print("Running the TelescopeOn command")
try:
    tmc_central_node.TelescopeOn()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not run command: {err_msg}")
print("Ready")

In [ ]:
tel_on_retry = widgets.IntSlider(
    value=20,
    min=0,
    max=100,
    step=1,
    description='Retry',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(tel_on_retry)
tel_on_wait = widgets.IntSlider(
    value=5,
    min=0,
    max=30,
    step=1,
    description='Sleep',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(tel_on_wait)

### Check if telescope is on

In [ ]:
# Check if telescope is on
nmax = tel_on_retry.value
sleep_time = tel_on_wait.value
p = widgets.IntProgress(
    value=0,
    min=0,
    max=nmax,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(p)
n = 0
while int(tmc_central_node.telescopeState) != 0:
    # print("Waiting for Telescope to come ON")
    n += 1
    p.value = n
    if n >= nmax:
        break
    try:
        time.sleep(sleep_time)
    except KeyboardInterrupt:
        print("Abort")
        break
    
tel_on = widgets.Valid(
    value=not tmc_central_node.telescopeState,
    description="Telescope ON",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(tel_on)

# Assign Resources

In [ ]:
# Assign Resources

print(f"SDP subarray is {sdp_subarray.state()}")
sdp_sub_w = widgets.Valid(
    value=sdp_subarray.state(),
    description="SDP subarray",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(sdp_sub_w)

time.sleep(3)

print(
    "Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS


# print(f"\nAssign_resources_json file contents: \n{assign_resources_json}")

try:
    tmc_subarray.AssignResources(json.dumps(assign_resources_json))
    res_ok = True
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not assign resources: {err_msg}")
    res_ok = False

time.sleep(2)
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

res_ok_w = widgets.Valid(
    value=res_ok,
    description="Resources",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(res_ok_w)

## Deploy dishes

In [ ]:
from astropy.time import Time
from tango import DeviceProxy

dish_deployments = [f"tango://tango-databaseds.{SKA001_NAMESPACE}.svc.miditf.internal.skao.int:10000/mid-dish/dish-manager/SKA001", 
                    f"tango://tango-databaseds.{SKA036_NAMESPACE}.svc.miditf.internal.skao.int:10000/mid-dish/dish-manager/SKA036"]


def get_tai_from_unix_s(unix_s: float) -> float:
    """
    Calculate atomic time in seconds from unix time in seconds.

    :param unix_s: Unix time in seconds

    :return: atomic time (tai) in seconds
    """
    astropy_time_utc = Time(unix_s, format="unix")
    return astropy_time_utc.unix_tai

dish_w = widgets.IntProgress(
    value=0,
    min=0,
    max=len(dish_deployments),
    description='Dish deploy',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(dish_w)

n = 0
for dish in dish_deployments:
    n += 1
    dish_w.value = n
    dish_manager_proxy = DeviceProxy(dish)
    current_pointing = dish_manager_proxy.achievedPointing
    current_az = current_pointing[1]
    current_el = current_pointing[2]

    current_time_tai_s = get_tai_from_unix_s(time.time() + 120)

    # Directions to move values
    az_dir = 1 if current_az < 350 else -1
    el_dir = 1 if current_el < 80 else -1

    track_table = [
        current_time_tai_s + 3,
        current_az + 1 * az_dir,
        current_el + 1 * el_dir,
        current_time_tai_s + 5,
        current_az + 2 * az_dir,
        current_el + 2 * el_dir,
        current_time_tai_s + 7,
        current_az + 3 * az_dir,
        current_el + 3 * el_dir,
        current_time_tai_s + 9,
        current_az + 4 * az_dir,
        current_el + 4 * el_dir,
        current_time_tai_s + 11,
        current_az + 5 * az_dir,
        current_el + 5 * el_dir,
    ]

    dish_manager_proxy.programTrackTable = track_table


## Start Scan

### Configure Scan

In [ ]:
# Configure Scan
# print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

# for key in configure_scan_json:
#     print(f"{key} {configure_scan_json[key]}")

# print(json.dumps(configure_scan_json))

show_textbox(configure_scan_json["pointing"]["target"]["target_name"], "Target")
show_textbox(str(configure_scan_json["tmc"]["scan_duration"]), "Duration")

try:
    tmc_subarray.Configure(json.dumps(configure_scan_json))
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not configure scan: {err_msg}")
    res_ok = False
    
time.sleep(12)
show_textbox(str(sdp_subarray_leaf_node_dp.sdpSubarrayObsState), "SDP Leaf")
show_textbox(str(csp_subarray_leaf_node_dp.cspSubarrayObsState), "CSP Leaf")


### Start scan

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(f'SDP interface {sdp_scan["interface"]}')
    try:
        sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))
    except tango.DevFailed as t_err:
        err_msg: str = t_err.args[0].desc.strip()
        print(f"Could not do SDP scan: {err_msg}\n")
        res_ok = False

time.sleep(10)
print(sdp_subarray_leaf_node_dp.sdpSubarrayObsState)

with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(f'CSP interface {csp_scan["interface"]}')
    try:
        csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))
    except tango.DevFailed as t_err:
        err_msg: str = t_err.args[0].desc.strip()
        print(f"Could not do CSP scan: {err_msg}\n")
        res_ok = False

time.sleep(2)
show_textbox(str(csp_subarray_leaf_node_dp.cspSubarrayObsState), "CSP Leaf")

### End scan

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
# print("Running the End Scan command: subarray obsstate should go to Ready state")

try:
    sdp_subarray_leaf_node_dp.EndScan()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not end SDP scan: {err_msg}\n")
time.sleep(2)
try:
    csp_subarray_leaf_node_dp.EndScan()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not end CSP scan: {err_msg}\n")
    
show_textbox(str(sdp_subarray_leaf_node_dp.sdpSubarrayObsState), "SDP Leaf ObsState")
show_textbox(str(csp_subarray_leaf_node_dp.cspSubarrayObsState), "CSP Leaf ObsState")


## End

### Run the End command

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print("Running the End command: subarray obsstate should go to Idle state")

try:
    sdp_subarray_leaf_node_dp.End()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not end SDP: {err_msg}")
time.sleep(2)
try:
    csp_subarray_leaf_node_dp.End()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not end CSP: {err_msg}")

time.sleep(2)
show_textbox(str(sdp_subarray_leaf_node_dp.sdpSubarrayObsState), "SDP Sub ObsState")
show_textbox(str(csp_subarray_leaf_node_dp.cspSubarrayObsState), "CSP Sub ObsState")


### Release All Resources

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

try:
    sdp_subarray_leaf_node_dp.ReleaseAllResources()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not relase SDP: {err_msg}")
time.sleep(2)
try:
    csp_subarray_leaf_node_dp.ReleaseAllResources()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not relase CSP: {err_msg}")

time.sleep(2)

show_textbox(str(sdp_subarray_leaf_node_dp.sdpSubarrayObsState), "SDP Sub ObsState")
show_textbox(str(csp_subarray_leaf_node_dp.cspSubarrayObsState), "CSP Sub ObsState")

### Run the Telescope Off command

In [ ]:
# Telescope Off
# print("Running the TelescopeOff command")

tel_off = False
try:
    tmc_central_node.TelescopeOff()
    tel_off = True
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not turn telescope off: {err_msg}")
    
tel_off_w = widgets.Valid(
    value=tel_off,
    description="Telescope OFF",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(tel_off_w)

time.sleep(5)
show_textbox(tmc_central_node.Status(), "TMC Central Node")
show_textbox(csp_control.Status(), "CSP Control")
show_textbox(cbf_controller.Status(), "CBF Controller")